In [1]:
# !pip install yelpapi
!pip install tqdm

In [2]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [3]:
# Load API Credentials
with open('/Users/sheelvohra/.secret/yelp_api.json') as f: 
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
# Instantiate YelpAPI instance
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [5]:
# Test API call
results = yelp_api.search_query(location='Chicago, IL', term='pizza')

In [6]:
type(results)

dict

In [7]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
# Set our API call parameters
LOCATION = "Chicago, IL 60622"
TERM = "thai"

In [9]:
# Specify folder for saving data
FOLDER = "data/"
os.makedirs(FOLDER , exist_ok=True)

In [10]:
# Obtain the city from the location
CITY = LOCATION.split(',')[0]

In [11]:
JSON_FILE = f"{FOLDER}{TERM}_{CITY}.json"
JSON_FILE

'data/thai_Chicago.json'

In [12]:
# Display the directory name of the JSON file
os.path.dirname(JSON_FILE)


'data'

In [15]:
# Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
# If it does not exist: 
if file_exists == False:
    # Create any needed folders
    # Get the folder name
    folder = os.path.dirname(JSON_FILE)
    # If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)   
    # Inform the user
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    # saving empty list
    with open(JSON_FILE,'w') as f:
        json.dump([], f)
# Advise if the file already exists
else:
    print(f"[i] {JSON_FILE} already exists")

[i] data/thai_Chicago.json already exists


In [16]:
# Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)

# Display the number of previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


# Testing

In [17]:
# Use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(term=TERM, location=LOCATION)

In [20]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [21]:
# Display the total number of results
results['total']


193

In [22]:
# Display the businesses datatype
type(results['businesses'])

list

In [23]:
# Calculate the number of results per page 
results_per_page = len(results['businesses'])
# Display the number of results per page  
results_per_page

20

In [24]:
# Calculate how many pages of results needed to cover the total_results
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil(results['total'] / results_per_page)
n_pages

10

In [26]:
# Loop through pages and append API call results to the JSON file
for i in tqdm_notebook(range(0, n_pages+1)):
    try:
        time.sleep(.2)
        # Load JSON file
        with open(JSON_FILE,'r') as f:
            previous_results = json.load(f)
        # Obtain current number of results in JSON file
        n_results = len(previous_results)
        # Obtain next page of results
        results = yelp_api.search_query(term=TERM, 
                                        location=LOCATION,
                                        offset=n_results+1)
        # Append this page of results and save to file
        previous_results.extend( results['businesses'])
        with open(JSON_FILE,'w') as f:
            json.dump(previous_results, f)
    # If there is an error exception, print the error message and break
    except Exception as e:
        print(e)
        break

  0%|          | 0/11 [00:00<?, ?it/s]

In [27]:
# Creat a dataframe for the JSON file
df = pd.read_json(JSON_FILE)
# View head and tail of dataframe
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,p2cCK4Vu-3L-skOMd93VVg,jj-thai-street-food-chicago,JJ Thai Street Food,https://s3-media4.fl.yelpcdn.com/bphoto/q8CXoP...,False,https://www.yelp.com/biz/jj-thai-street-food-c...,399,"[{'alias': 'thai', 'title': 'Thai'}]",4.5,"{'latitude': 41.8958251, 'longitude': -87.6703...","[delivery, pickup]",$$,"{'address1': '1715 W Chicago Ave', 'address2':...",+13129294435,(312) 929-4435,1612.454741
1,kuJLjqq0m_tWfkgkMV6WrA,sweet-rice-asian-kitchen-chicago,Sweet Rice Asian Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/-YeUNP...,False,https://www.yelp.com/biz/sweet-rice-asian-kitc...,28,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",5.0,"{'latitude': 41.915980361557416, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '1904 N Western Ave', 'address2':...",+17736612743,(773) 661-2743,1383.159973
2,bg20bWGO5cD7A873sEb9kw,silom-12-chicago,Silom 12,https://s3-media4.fl.yelpcdn.com/bphoto/6-ueD1...,False,https://www.yelp.com/biz/silom-12-chicago?adju...,474,"[{'alias': 'thai', 'title': 'Thai'}]",4.0,"{'latitude': 41.91487, 'longitude': -87.68488}","[pickup, delivery]",$$,"{'address1': '1846-48 N Milwaukee Ave', 'addre...",+17734891212,(773) 489-1212,1272.165974
3,c-mWMo-TcHiVrr6_OenOxg,noble-thai-chicago,Noble Thai,https://s3-media4.fl.yelpcdn.com/bphoto/_e64FP...,False,https://www.yelp.com/biz/noble-thai-chicago?ad...,53,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.5,"{'latitude': 41.8959, 'longitude': -87.66216}","[pickup, delivery]",NaN,"{'address1': '1371 W Chicago Ave', 'address2':...",+13128461867,(312) 846-1867,2209.580396
4,VH6O_2qSPkTme_s-1Y1nXQ,three-wheels-noodle-chicago,Three Wheels Noodle,https://s3-media2.fl.yelpcdn.com/bphoto/WJC6ch...,False,https://www.yelp.com/biz/three-wheels-noodle-c...,139,"[{'alias': 'thai', 'title': 'Thai'}]",4.5,"{'latitude': 41.92396, 'longitude': -87.69839}","[pickup, delivery]",$$,"{'address1': '2339 N Milwaukee Ave', 'address2...",+17733608288,(773) 360-8288,2457.241713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,pqO59Dor8BZGJsDvlJshrQ,new-tokyo-chicago,New Tokyo,https://s3-media1.fl.yelpcdn.com/bphoto/P7i3EM...,False,https://www.yelp.com/biz/new-tokyo-chicago?adj...,221,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",3.5,"{'latitude': 41.93895, 'longitude': -87.64414}","[delivery, pickup]",$$,"{'address1': '3139 N Broadway St', 'address2':...",+17732481193,(773) 248-1193,5279.559180
188,Brl55B0WFo3N7YzprXrvSw,panera-bread-chicago-7,Panera Bread,https://s3-media2.fl.yelpcdn.com/bphoto/gjC5iw...,False,https://www.yelp.com/biz/panera-bread-chicago-...,127,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.0,"{'latitude': 41.86877712736723, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '1101 South Canal Street', 'addre...",+13127861761,(312) 786-1761,5553.555467
189,kr2ipJbSnYUYI0fdwQizaw,soupbox-chicago-4,Soupbox,https://s3-media2.fl.yelpcdn.com/bphoto/su3_u-...,False,https://www.yelp.com/biz/soupbox-chicago-4?adj...,610,"[{'alias': 'soup', 'title': 'Soup'}, {'alias':...",3.5,"{'latitude': 41.936229, 'longitude': -87.644115}","[delivery, pickup]",$$,"{'address1': '2943 N Broadway St', 'address2':...",+17739359800,(773) 935-9800,5070.702717
190,bnIJC1iAHGq0VBWCjIvR7g,panera-bread-chicago-14,Panera Bread,https://s3-media1.fl.yelpcdn.com/bphoto/ZoPWLP...,False,https://www.yelp.com/biz/panera-bread-chicago-...,92,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.0,"{'latitude': 41.90404, 'longitude': -87.638714}","[delivery, pickup]",$,"{'address1': '400 West Division Street', 'addr...",+13123372304,(312) 337-2304,3985.111724


In [28]:
df.duplicated(subset='id').sum()

0

In [29]:
# Convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
# Display the file name
csv_file

'data/thai_Chicago.csv.gz'

In [32]:
# Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression='gzip', index=False)

In [33]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(csv_file)

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')
print('\n')
print(f'The csv.gz is {size_json/size_csv_gz:.2f} times smaller!')

JSON FILE: 186,436 Bytes
CSV.GZ FILE: 26,017 Bytes


The csv.gz is 7.17 times smaller!
